In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import wignerDistribution as wd

## Compute the 1D PWD

**Load dictionaries** 

In [2]:
def loadMatFile(file_path, file_name, key):
    """
    Load a mat file and return an item of the dictionary loaded.
    """    
    # read mat file dictionary
    dictionary = scipy.io.loadmat(file_path + file_name)
    
    # access item of a dictionary
    array = dictionary[key]
    
    return array

In [3]:
# Folder's path
path = 'C:\\Users\\ferna\\Desktop\\computerGeneratedHolography'

# Load hologram dataset
file_path = path + '\\output\\dataset\\'
file_name = 'hDataset.mat'
key = 'hDataset'
hologram_dataset = loadMatFile(file_path, file_name, key)

# Load reconstructed images dataset
file_name = 'rDataset.mat'
key = 'rDataset'
reconstruction_dataset = loadMatFile(file_path, file_name, key)

# Load 3D points dataset
file_name = 'pDataset.mat'
key = 'pDataset'
points_dataset = loadMatFile(file_path, file_name, key)

In [4]:
print(hologram_dataset.shape)
print(reconstruction_dataset.shape)
print(points_dataset.shape)

(200, 200, 25)
(200, 200, 25)
(75, 3)


**Compute 1D Pseudo-Wigner Distribution**

In [5]:
def calculate_1D_wigner_distribution(dataset, n, seq_legth, angle):
    """
    Calculates the  1D pseudo-Wigner distribution of the n first images (in gray levels) in the dataset. seq_length is the 
    length in pixels of the operating window and it has to be an odd number (9 is a common operative value). 
    The angle variable in degrees determines the spatial orientation of the distribution.
    """
    wd_results = []
    for i in range(n):
        test_image = dataset[:,:,i]
        wd_results.append(wd.wigner_distribution(test_image, seq_legth, angle))
    return wd_results

In [20]:
%%time

dataset = hologram_dataset
n = hologram_dataset.shape[2]
seq_legth = 9 # length in pixels of the operating window
angle = 0     # spatial orientation of the distribution (degrees)

wd_results = calculate_1D_wigner_distribution(dataset, n, seq_legth, angle)

calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  100  ...
calculating in row  200  ...
calculating ...
calculating in row  10

In [240]:
# Save to .npy file
np.save('wd_results', wd_results)

In [241]:
# Load .npy file
wd_results_array = np.load('wd_results.npy')
wd_results_array.shape

(25, 8, 200, 200)

## Compute database to Machine Learning algorithms

**Functions to be used in the features' computation**

The functions below compute the sobel of an image and the count the number of pixels in the x and y axis.

In [36]:
def sobel(image):
    w = len(image)
    kernel_x = np.array([ [ 1, 0,-1],
                          [ 2, 0,-2],
                          [ 1, 0,-1] ])

    kernel_y = np.array([ [ 1, 2, 1],
                          [ 0, 0, 0],
                          [-1,-2,-1] ])
    
    grad_x = np.zeros([w - 2, w - 2])
    grad_y = np.zeros([w - 2, w - 2])
    
    for i in range(w - 2):
        for j in range(w - 2):
            grad_x[i, j] = sum(sum(image[i : i + 3, j : j + 3] * kernel_x))
            grad_y[i, j] = sum(sum(image[i : i + 3, j : j + 3] * kernel_y))
            if grad_x[i, j] == 0:
                grad_x[i, j] = 0.000001 
    
    mag = np.sqrt(grad_y ** 2 + grad_x ** 2)
    ang = np.arctan(grad_y / (grad_x + np.finfo(float).eps))
  
    # Gradient computation
    return [mag,ang]

def pixel_count(image):
    pc_x = np.zeros(len(image))
    pc_y = np.zeros(len(image))
  
    # Pixel count computation
    for i in range(len(image)):
        pc_x[i] = np.count_nonzero(image[i, :])
        pc_y[i] = np.count_nonzero(image[:, i])

    return [pc_x, pc_y]

**Class for the set of images generated by the Wigner Distribution of an image**

The 1D PWD (Pseudo-Wigner Distribution) generates 8 imagens from one image as input.

In [250]:
class Element:
    def __init__(self, data, target):
        self.target = target
        self.features = {'var' : 0,
                         'std' : 0,
                         'mean_grad_M' : 0,
                         'std_grad_M'  : 0,
                         'mean_grad_D' : 0,
                         'std_grad_D'  : 0,
                         'mean_PC_X'   : 0,
                         'std_PC_X'    : 0,
                         'active_PC_X' : 0,
                         'mean_PC_Y'   : 0,
                         'std_PC_Y'    : 0,
                         'active_PC_Y' : 0}
        self.computeFeatures(data)
        
    def computeFeatures(self, data):
        """
        Compute the 12 features of each image (8 images in total) and returns the average of each feature.
        """
        # Auxiliar variable
        matrix = np.zeros((8,12))
        pos = 0 # 0..7
        
        # Feature computation for each image
        for image in data:
            # Feature computation
            mag, ang = sobel(image)
            pcx, pcy = pixel_count(image)
            matrix[pos][0] = (np.var(image))
            matrix[pos][1] = (np.std(image))
            matrix[pos][2] = (np.mean(mag))
            matrix[pos][3] = (np.std(mag))
            matrix[pos][4] = (np.mean(ang))
            matrix[pos][5] = (np.std(ang))
            matrix[pos][6] = (np.mean(pcx))
            matrix[pos][7] = (np.std(pcx))
            matrix[pos][8] = (np.count_nonzero(pcx))
            matrix[pos][9] = (np.mean(pcy))
            matrix[pos][10] = (np.std(pcy))
            matrix[pos][11] = (np.count_nonzero(pcy)) 
            # Update variable
            pos = pos + 1
        
        # Features' average
        keys = ['var', 'std', 'mean_grad_M', 'std_grad_M', 'mean_grad_D', 'std_grad_D', 'mean_PC_X', 'std_PC_X', 'active_PC_X', 'mean_PC_Y', 'std_PC_Y', 'active_PC_Y']
        for i in range(12):
            self.features[keys[i]] = sum(matrix[:,i]/8)
    
    def __print__(self):
        print("Element target: " + str(self.target))
        print("Element features:")
        print(self.features)

In [253]:
'''array = wd_results_array[2]
print(array.shape)

elem = Element(array, 1)
elem.__print__()
'''

(8, 200, 200)
Element target: 1
Element features:
{'var': 9024828.034729265, 'std': 2430.215837108408, 'mean_grad_M': 10334.639101530625, 'std_grad_M': 6843.350135753646, 'mean_grad_D': 0.02804709551718424, 'std_grad_D': 0.8803386817979355, 'mean_PC_X': 200.0, 'std_PC_X': 0.0, 'active_PC_X': 200.0, 'mean_PC_Y': 200.0, 'std_PC_Y': 0.0, 'active_PC_Y': 200.0}


**Class for create a dataset with all images computed by Octave's code**

In [273]:
class Dataset:
    def __init__(self, array, length, y_list):
        self.array = array   # (nb_examples, 8, 200, 200)
        self.length = length # nb_examples
        self.wd_list = []
        self.wd_list = self.createElements(y_list)
        self.features = [[float(f) for f in elem.features.values()] for elem in self.wd_list]
        self.raw_targets  = [[self.wd_list[i].target] for i in range(self.length)]
    
    def createElements(self, y_list):
        elements = []
        pos = 0
        for row in self.array:
            # row : (8, 200, 200)
            elements.append(Element(row, y_list[pos]))
            pos = pos + 1
        return elements

In [274]:
def computeTargets(array):
    targets = []
    y = 1
    for i in range(len(array)):
        targets.append(y)
        if((i+1)%5 == 0):
            y = y + 1
    return targets

In [277]:
'''
array = wd_results_array[0:2, :, :, :]
target_list = computeTargets(wd_results)

test = Dataset(array, len(array), target_list)
print(test.array.shape)
test.wd_list[0].__print__()
test.wd_list[1].__print__()
'''

(2, 8, 200, 200)
Element target: 1
Element features:
{'var': 10218840.74310089, 'std': 2753.179664325147, 'mean_grad_M': 9800.17669808202, 'std_grad_M': 6686.825004976745, 'mean_grad_D': 0.4245315875688738, 'std_grad_D': 0.6320330591719492, 'mean_PC_X': 200.0, 'std_PC_X': 0.0, 'active_PC_X': 200.0, 'mean_PC_Y': 200.0, 'std_PC_Y': 0.0, 'active_PC_Y': 200.0}
Element target: 1
Element features:
{'var': 9828456.360765265, 'std': 2491.277620935381, 'mean_grad_M': 9278.490214573858, 'std_grad_M': 6634.414920994312, 'mean_grad_D': 0.0045929823100968005, 'std_grad_D': 0.8127970953675628, 'mean_PC_X': 200.0, 'std_PC_X': 0.0, 'active_PC_X': 200.0, 'mean_PC_Y': 200.0, 'std_PC_Y': 0.0, 'active_PC_Y': 200.0}


In [283]:
def generate_dataset(array):
    """
    bla bla bla
    """
    target_list = computeTargets(array)
    dataset = Dataset(array, len(array), target_list)
    
    return dataset

def cvt_obj_nparray(dataset):
    X = np.zeros((dataset.length, 12))
    Y = np.zeros((dataset.length,))
    for i, elem in enumerate(dataset.wd_list):
        Y[i] = elem.target
        for j, feature in enumerate(elem.features):
            X[i, j] = elem.features[feature]
    return X, Y

def create_data_file(filename):
    # Load the database (.npy) files 
    wd_results_array = np.load(filename) 

    print("Creating dataset...")
    data_set = generate_dataset(wd_results_array)
    print ("\nFinished creating dataset\n")

    X_array, Y_array = cvt_obj_nparray(data_set)

    return X_array, Y_array

#Function that normalize the features
def normalize(arr):
    max_line = np.max(arr, axis=0)
    min_line = np.min(arr, axis=0)
    
    arr = (arr - min_line) / (max_line - min_line)
    
    return arr

In [284]:
%%time
array = wd_results_array[0:5, :, :, :]
dataset = generate_dataset(array)

Wall time: 40.6 s


In [285]:
dataset.length

5

In [287]:
%%time

X_array, Y_array = create_data_file('wd_results.npy')

Creating dataset...

Finished creating dataset

Wall time: 3min 37s


In [292]:
print(X_array.shape)
print(Y_array.shape)

(25, 12)
(25,)
